In [ ]:
import tensorflow as tf
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras.layers import Conv2D, Input, BatchNormalization, LeakyReLU, ZeroPadding2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

YOLOv3는 3개의 scale을 가지는 feature map을 통해 object detection을 수행합니다. 각 scale에서는 3개의 anchor box를 사용하므로 총 9개의 anchor box를 사용합니다. 먼저, 입력 이미지를 416x416 크기로 resize한 후에, Darknet-53 네트워크를 통과시켜 feature map을 추출합니다. 이후, 추출된 feature map을 이용하여 object detection을 수행합니다.

In [ ]:
# 원본 이미지가 들어갈 리스트 만들기
images_originals = []

# 원본 이미지가 저장된 폴더에서 하나씩 img 불러서 리스트에 담기
for img in glob.glob('../Semi_Data/images/*.png') :
    images_originals.append(mpimg.imread(img))

annotation_file = []



In [ ]:
def get_annotation_filename(image_filename):
    return image_filename.replace('.jpg', '.txt').replace('.jpeg', '.txt')

def read_annotation(annotation_filename):
    with open(annotation_filename, 'r') as f:
        lines = f.readlines()

    objects = []
    for line in lines:
        parts = line.strip().split(' ')
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        objects.append({'class_id': class_id, 'x_center': x_center, 'y_center': y_center, 'width': width, 'height': height})

    return objects


def load_image_and_labels(image_filename):
    # 이미지 파일 읽기
    image = tf.io.read_file(image_filename)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # 어노테이션 파일 읽기
    annotation_filename = get_annotation_filename(image_filename)
    labels = read_annotation(annotation_filename)
    
    return image, labels



In [ ]:
# 1. 데이터 준비
train_dataset = tf.data.Dataset.from_tensor_slices(train_filenames)
train_dataset = train_dataset.map(load_image_and_labels)

val_dataset = tf.data.Dataset.from_tensor_slices(val_filenames)
val_dataset = val_dataset.map(load_image_and_labels)

# 2. 모델 빌드
model = build_yolov3_model()

# 3. 모델 학습
model.compile(optimizer=optimizer, loss=loss_fn)

history = model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

# 4. 모델 평가
test_dataset = tf.data.Dataset.from_tensor_slices(test_filenames)
test_dataset = test_dataset.map(load_image_and_labels)

results = model.evaluate(test_dataset)

# 5. 모델 추론
image = load_image(image_path)
image = tf.expand_dims(image, 0)
pred = model.predict(image)
boxes, scores, classes, num_detections = tf.split(pred, [4, 1, num_classes, 1], axis=-1)